# Chapter 12: Signaling adversity

## Determine the kinds

In [6]:
id :: a -> a
-- a is *

: 

In [7]:
r :: a -> f a
-- a is *
-- f is * -> *

: 

## String processing

In [8]:
replaceThe :: String -> String
replaceThe = unwords . fmap normalize . words where
    normalize "the" = "a"
    normalize s = s
    
replaceThe "the cow loves the us"

"a cow loves a us"

In [9]:
countTheBeforeVowel :: String -> Int
countTheBeforeVowel [] = 0
countTheBeforeVowel str = length vs where
    ws = words str
    ps = zip ws (tail ws)
    thes = filter ((== "the") . fst) ps
    vs = filter ((`elem` "eyuioa") . head . snd) thes
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [34]:
countTheBeforeVowel :: String -> Int
countTheBeforeVowel = length . filter ((`elem` "eyuioa") . head . snd) . filter ((== "the") . fst) . zipWithRest . words where
    zipWithRest xs = zip xs (tail xs)
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [46]:
class Truthy a where
    isTruthy :: a -> Bool
    
instance (Truthy a, Truthy b) => Truthy (a, b) where
    isTruthy (a, b)
        | isTruthy a && isTruthy b = True
        | otherwise = False
        
instance Truthy Bool where
    isTruthy x = x

instance Truthy a => Truthy (Maybe a) where
    isTruthy Nothing = False
    isTruthy (Just x) = isTruthy x
    
isTruthy True
isTruthy False
isTruthy (True, True)
isTruthy (True, False)

isTruthy (Nothing :: Maybe Bool)
isTruthy (Just True)
isTruthy (Just False)

True

False

True

False

False

True

False

In [48]:
import Data.Bifunctor (bimap)

safeHead :: [a] -> Maybe a
safeHead [] = Nothing
safeHead (x:_) = Just x

isVowel :: Char -> Bool
isVowel = (`elem` "eyuioa")

countTheBeforeVowel :: String -> Int
countTheBeforeVowel = length . filter isProperPair . zipWithRest . words where
    isVowelWord = isTruthy . fmap isVowel . safeHead
    isThe = (== "the")
    isProperPair = isTruthy . bimap isThe isVowelWord
    zipWithRest xs = zip xs (tail xs)
    
countTheBeforeVowel ""
countTheBeforeVowel "hello"
countTheBeforeVowel "hello the"
countTheBeforeVowel "hello the qwer"
countTheBeforeVowel "hello the awer the basdf the aaa"

0

0

0

0

2

In [51]:
countVowels :: String -> Int
countVowels = length . filter isVowel

countVowels "qweraa"

3

## Validate the word

In [53]:
newtype Word' = Word' String deriving (Eq, Show)

mkWord :: String -> Maybe Word'
mkWord str
    | numVowels > numConsonants = Nothing
    | otherwise = Just $ Word' str
    where
        numVowels = countVowels str
        numConsonants = length str - numVowels
        
mkWord "qwer"
mkWord "aaeass"

Just (Word' "qwer")

Nothing

## It's only natural

In [55]:
data Nat
    = Zero
    | Succ Nat
    deriving (Eq, Show)
    
natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ n) = 1 + natToInteger n

natToInteger Zero
natToInteger (Succ Zero)
natToInteger (Succ (Succ Zero))

0

1

2

In [59]:
integerToNat :: Integer -> Maybe Nat
integerToNat n
    | n < 0 = Nothing
    | n == 0 = Just Zero
    | Just p <- integerToNat (n - 1) = Just $ Succ p
    | otherwise = Nothing

integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))

In [66]:
integerToNat :: Integer -> Maybe Nat
integerToNat int
    | int < 0 = Nothing
    | otherwise = Just $ f int
    where
        f 0 = Zero
        f n = Succ $ f (n - 1)
        
integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3        

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))

In [73]:
import Control.Applicative (Alternative)
import Control.Monad (guard)

ensure :: Alternative f => (a -> Bool) -> a -> f a
ensure p a = a <$ guard (p a)

(ensure (> 10) 123) :: Maybe Int
(ensure (> 10) 5) :: Maybe Int

Just 123

Nothing

In [75]:
:t (<$)

(<$) :: forall (f :: * -> *) a b. Functor f => a -> f b -> f a

In [76]:
:t guard

guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

In [79]:
integerToNat :: Integer -> Maybe Nat
integerToNat = fmap f . ensure (>= 0) where
    f 0 = Zero
    f n = Succ $ f (n - 1)
        
integerToNat (-10)
integerToNat 0
integerToNat 1
integerToNat 3        

Nothing

Just Zero

Just (Succ Zero)

Just (Succ (Succ (Succ Zero)))